# MySQL profiling 
This notebook will first load actual papers from a formatted XML, then start profiling with these data on MySQL database.
Each row in the table represents one paper

## Load data from XML

In [ ]:
# by yixuan
import os
import random
import time
from tqdm import tqdm

import bs4, lxml
from bs4 import BeautifulSoup as bs

import mysql.connector

In [ ]:
def get_timestamp():
    return f'{time.time()}'

def generate_ids(num, total):
    randomlist = []
    for i in range(num):
        randomlist.append(random.randint(0, total-1))
    return randomlist

class XMLLoader:
    """
    Load XML files into bs4 content
    """
    def __init__(self):
        self.xmls = []
        self.items = []
    
    def read_xmls(self, xmls_path):
        """
        xmls_path contains all the xmls files's paths
        one line for one xml path
        each xml file will be parsed and stored as a BeautifulSoup
        parsed object in self.xmls
        """
        print('loading...')
        with open(xmls_path, 'r') as f:
            for xml_path in f:
                self.xmls.append(self._read_one_xml(xml_path))
                
        # serialize items, just load it to the memory
        for bs_content in tqdm(self.xmls):
            for item in bs_content.find_all('item'):
                title = item.title.get_text()
                abstract = item.abstract.get_text()
                author = item.author.get_text()
                url = item.url.get_text()
                self.items.append((title, abstract, author, url))
                
    
    def _read_one_xml(self, xml_path):
        """
        return the BeautifulSoup parsed result of a given xml file
        """
        xml_path = xml_path.strip()
        with open(xml_path, 'r') as f:
            content = f.read()
            bs_content = bs(content, 'lxml')
            return bs_content


In [ ]:
xml_loader = XMLLoader()
xml_loader.read_xmls('xml_paths')

## MySQL profiling


In [ ]:
# Following are the data structures useful for handling with DB
class DatabaseHandler:
    def __init__(self, items):
        self.items = items
        
    def connect(self):
        """
        connect to a db, return the connect result
        """
        pass
        
    def load_to_db(self, num=1000):
        """
        insertion test
        randomly sample num xml items and upload them 
        to db along with the timestamp, return the total 
        time usage 
        
        """
        pass
    
    
class MySQL(DatabaseHandler):
    def __init__(self, items, host, user, password, db_name):
        super().__init__(items)
        self.host = host
        self.user = user
        self.password = password
        self.db_name = db_name
        self.db = None
    
    def _print_cursor(self, cursor):
        for x in cursor:
            print(x)
        
    def connect(self):
        self.db = mysql.connector.connect(
            host = self.host, 
            user = self.user, 
            password = self.password, 
            database = self.db_name
        )
        print(self.db)
    
    def create_table(self, table_name):
        """
        create a test table
        """
        
        self.table_name = table_name
        sql = f'''CREATE TABLE {self.table_name} (
            title VARCHAR(255) NOT NULL,
            abstract VARCHAR(4096) NOT NULL, 
            author VARCHAR(512) NOT NULL, 
            url VARCHAR(512) NOT NULL, 
            timestamp VARCHAR(64) NOT NULL, 
            id INT PRIMARY KEY AUTO_INCREMENT)
            '''
        cursor = self.db.cursor()
        cursor.execute(f'DROP TABLE IF EXISTS {self.table_name}')
        self._print_cursor(cursor)
        cursor.execute(sql)
        self._print_cursor(cursor)
    
    
    def show_tables(self):
        cursor = self.db.cursor()
        cursor.execute('SHOW TABLES')
        self._print_cursor(cursor)
    
    def show_k_rows(self, table, k=5):
        cursor = self.db.cursor()
        cursor.execute(f'SELECT * FROM {table} LIMIT {k}')
        self._print_cursor(cursor)
        
    def load_to_db(self, num, table):
        total = len(self.items)
        randomlist = generate_ids(num, total)
        cursor = self.db.cursor()
        for i in tqdm(randomlist):
            sql = f'''INSERT INTO {table}(
                title, abstract, author, url, timestamp) 
                VALUES (%s, %s, %s, %s, %s)'''
            basic_item = self.items[i]
            basic_item += (get_timestamp(), )
            cursor.execute(sql, basic_item)
            self.db.commit()
        
        

In [ ]:
# connect to db and create an empty table named 'small test'
table = 'small_test'
mysql_loader = MySQL(xml_loader.items, 'localhost', 'ttt', 'password@ttt', 'ttt_db_efficiency')
mysql_loader.connect()

In [ ]:
mysql_loader.create_table(table)
mysql_loader.show_tables()

('papers',)
('small_test',)


### Writing Test
write 1m papers to db

In [ ]:
# randomly sample 1m papers and load them to the db
mysql_loader.load_to_db(1000000, table)

In [ ]:
# show the first k results
mysql_loader.show_k_rows(table)

('RSR2015: database for text-dependent speaker verification using multiple pass-phrases', 'This paper describes a new speech corpus, the RSR2015 database designed for text-dependent speaker recognition with scenario based on fixed pass-phrases. This database consists of over 71 hours of speech recorded from English speakers covering the diversity of accents spoken in Singapore. Acquisition has been done using a set of six portable devices including smart phones and tablets. The pool of speakers consists of 298 participants (142 female and 156 male speakers) from 17 to 42 years old. We propose a protocol for the case of user-dependent passphrases in text-dependent speaker recognition and we also report speaker recognition experiments on RSR2015 database.\n', 'Anthony Larcher, Kong Aik Lee, Bin Ma, Haizhou Li', 'https://www.isca-speech.org/archive/pdfs/interspeech_2012/larcher12_interspeech.pdf', '1640363508.2227705', 1)
('Phonological Markers of Oxytocin and MDMA Ingestion', 'Speech dat

### Query Test
- Firstly query whether a data point exists, e.g., whether a title exists;
- Secondly query whether a word exists in the content of a data point, e.g. whether a keyword exists in the abstract

In [ ]:
# query test

class Querier:
    def __init__(self, items, db):
        self.items = items
        self.db = db
        self.vocab = []
        self.name = None
        self.idx = None
    
    def prepare_query_list(self, idx):
        # item structure
        # self.items.append((title, abstract, author, url))
        vocab = set()
        for line in self.items:
            line = line[idx].split()
            vocab = vocab.union(set(line))
        self.vocab = list(vocab)
    
    def single_query(self, num, table):
        """
        whether a tag exist
        """
        total = len(self.items)
        randomlist = generate_ids(num, total)
        elapsed = 0
        cursor = self.db.cursor()
        for i in randomlist:
            sql = f'SELECT * FROM {table} WHERE {self.name} = %s'
            tic = time.perf_counter()
            cursor.execute(sql, (self.items[i][self.idx],))
            cursor.fetchall()
            toc = time.perf_counter()
            elapsed += toc-tic
        return elapsed
    
    def keyword_query(self, num, table):
        """
        whether a keyword exists in the content
        """
        total = len(self.items)
        randomlist = generate_ids(num, total)
        elapsed = 0
        cursor = self.db.cursor()
        for i in randomlist:
            sql = f'SELECT * FROM {table} WHERE {self.name} LIKE %s'
            tic = time.perf_counter()
            cursor.execute(sql, ('%'+self.vocab[i]+'%',))
            cursor.fetchall()
            toc = time.perf_counter()
            elapsed += toc-tic
        return elapsed
    
    
class TitleQuerier(Querier):
    def __init__(self, items, db):
        super().__init__(items, db)
        self.idx = 0
        self.name = 'title'
        self.prepare_query_list(self.idx)
        
class AbstractQuerier(Querier):
    def __init__(self, items, db):
        super().__init__(items, db)
        self.idx = 1
        self.name = 'abstract'
        self.prepare_query_list(self.idx)
        
class AuthorQuerier(Querier):
    def __init__(self, items, db):
        super().__init__(items, db)
        self.idx = 2
        self.name = 'url'
        self.prepare_query_list(self.idx)
        
class UrlQuerier(Querier):
    def __init__(self, items, db):
        super().__init__(items, db)
        self.idx = 0
        self.name = 'url'
        self.prepare_query_list(self.idx)

In [ ]:
title_query = TitleQuerier(mysql_loader.items, mysql_loader.db)


In [ ]:
def profile_query(test_count, query_func, num_per_time, table):
    test_time = 0
    for i in tqdm(range(test_count)):
        test_time += query_func(num_per_time, table)
    print(test_time/test_count)

In [ ]:
num_of_test = 5
query_per_test = 1000
profile_query(num_of_test, title_query.single_query, query_per_test, table)

100%|█████████████████████████████████████████████| 5/5 [00:05<00:00,  1.15s/it]

1.1363654257263989


In [ ]:
num_of_test = 5
query_per_test = 1000
profile_query(num_of_test, title_query.keyword_query, query_per_test, table)

100%|█████████████████████████████████████████████| 5/5 [00:10<00:00,  2.11s/it]

2.0957737660501152
